In [1]:
import config
import pymongo
import numpy as np
import pandas as pd
import json
import re

import sqlalchemy as db
from sqlalchemy.orm import sessionmaker
from sqlalchemy import create_engine, inspect
from sqlalchemy.engine import reflection

In [2]:
myclient = pymongo.MongoClient(str(config.MONGO_URI))
stagingDb = myclient[str(config.DB_STAGING)]
analyseDb = myclient[str(config.DB_ANALYSE)]
stagingCol = stagingDb[config.COLL_PLAATJES]
stagingOud = stagingDb[config.COLL_STAGING_OUD]
stagingNieuw = stagingDb[config.COLL_STAGING_NIEUW]
analyseCol = analyseDb[config.COLL_ANALYSE]
analyseColClean = analyseDb[config.COLL_ANALYSE_CLEAN]

metaCollection = stagingDb['Kolominformatie']

lst_soorten_meta = ['Metaal',
 'Artefact',
 'Glas',
 'Stelling',
 'Hout',
 'Spijker',
 'Plaatsing',
 'Keramiek',
 'Put',
 'Vondst',
 'Standplaats',
 'Aardewerk',
 'Munt',
 'Muur',
 'Skelet',
 'Doos',
 'Project',
 'Spoor',
 'Vindplaats',
 'Leer',
 'Steen',
 'Bot',
 'Vlak',
 'Kleipijp']
df_soorten_meta = pd.DataFrame(lst_soorten_meta).rename(columns={0: 'Soort'}) 

In [3]:
def AggregateData(df, fase): 
    df['teller'] = pd.to_numeric(df['teller'])
    df = df.groupby('soort').agg({'project':lambda x: list(x), 'teller':lambda x: sum(x)})
    df['Count'] = df.apply(lambda x: len(x.project), axis=1)
    df['Stage'] = fase
    df = df.sort_values('teller', ascending=False).reset_index()
    df = df.rename(columns={'soort': 'Soort', 'project': 'Projecten', 'teller': 'Aantal_Records', 'Count': 'Aantal_projecten'}).sort_index()
    return df

In [4]:
grp_aggr = [{'$group': {'_id': {'project': "$brondata.project", 'soort': "$soort"},'teller': {"$sum": 1}}}
           ,{'$replaceRoot': {'newRoot': {'project': "$_id.project", 'soort': "$_id.soort", 'teller': { '$ifNull': ["$teller", 0]}}}}]

df_singlestore = AggregateData(pd.DataFrame(list(analyseCol.aggregate(grp_aggr))), 'SingleStore')
df_singlestore.head(5)

,Soort,Projecten,Aantal_Records,Aantal_projecten,Stage
0,Standplaats,[MAGAZIJN],8407,1,SingleStore
1,Aardewerk,"[DC112, DB34, DC97, DC72, DC36]",6953,5,SingleStore
2,Plaatsing,[MAGAZIJN],5634,1,SingleStore
3,Vondst,"[DC112, DB34, DC97, DC93, DC154, DC179, DC11]",4521,7,SingleStore
4,Doos,[MAGAZIJN],2773,1,SingleStore


In [5]:
grp_aggr_artef = [{"$match": {"soort": 'Artefact'}}
                 ,{'$group': {'_id': {'project': "$brondata.project", 'soort': "$artefactsoort"},'teller': {"$sum": 1}}}
                 ,{'$replaceRoot': {'newRoot': {'project': "$_id.project", 'soort': "$_id.soort", 'teller': { '$ifNull': ["$teller", 0]}}}}]

df1 = pd.DataFrame(list(analyseColClean.aggregate(grp_aggr_artef)))
df2 = pd.DataFrame(list(analyseColClean.aggregate(grp_aggr)))

df_singlestoreclean = AggregateData(df1.append(df2[df2.soort != 'Artefact']), 'SingleStoreClean')
df_singlestoreclean.head(5)

,Soort,Projecten,Aantal_Records,Aantal_projecten,Stage
0,Standplaats,[MAGAZIJN],8407,1,SingleStoreClean
1,Aardewerk,"[DC36, DB34, DC97, DC72, DC112]",6953,5,SingleStoreClean
2,Plaatsing,[MAGAZIJN],5634,1,SingleStoreClean
3,Vondst,"[DC179, DC112, DC11, DB34, DC97, DC93, DC154]",4521,7,SingleStoreClean
4,Doos,[MAGAZIJN],2773,1,SingleStoreClean


In [6]:
lst_tables = ['Def_Project', 'Def_Vondst', 'Def_Stelling', 'Def_Plaatsing', 'Def_Vindplaats', 'Def_Artefact']
regexProject = re.compile(r'\'project\': \'(.*?)\'') # regex to replace Object

def getProject(brondata):    
    #print(brondata)
    if brondata is None or brondata != "":
        return regexProject.search(brondata).group(1)
    else: 
        return "" 

df_tables_projects = pd.DataFrame()
    
engine = create_engine(config.SQLALCHEMY_DATABASE_URI)
with engine.connect() as connection:
    for table in lst_tables:     
        df = pd.read_sql_query('SELECT brondata from "' + table + '"', connection)
        
        df_out = pd.DataFrame()
        df_out['project'] = df.apply(lambda x: getProject(x['brondata']), axis=1)
        df_out['soort'] = table[4:]
        df_tables_projects = pd.concat([df_tables_projects, pd.DataFrame(df_out).groupby(['project', 'soort']).size().reset_index(name='teller')])
    
df_doelsysteem = AggregateData(df_tables_projects, 'Doelsysteem')
df_doelsysteem.head(5)


,Soort,Projecten,Aantal_Records,Aantal_projecten,Stage
0,Plaatsing,[MAGAZIJN],5634,1,Doelsysteem
1,Vondst,"[DB34, DC11, DC112, DC154, DC179, DC93, DC97]",4521,7,Doelsysteem
2,Project,[DELF-IT],840,1,Doelsysteem
3,Artefact,"[DC154, DC179]",175,2,Doelsysteem
4,Vindplaats,[DELF-IT],37,1,Doelsysteem


## Analyse van alle gegevens

Hieronder volgt een overzicht van alle gegevens die in de vier fases beschikbaar zijn. Zo kunnen fouten in de conversie worden opgespoord. 

In [7]:


def getOverview(OnWhat): 
    lst_countColumns = [OnWhat + '_singlestore', OnWhat + '_singlestoreclean', OnWhat + '_doelsysteem']
    
    df = df_singlestore[['Soort', 'Projecten', OnWhat]].merge(df_singlestoreclean[['Soort', OnWhat]], on=['Soort'], how='outer', suffixes=("_singlestore", "_singlestoreclean"))
    df = df.merge(df_doelsysteem[['Soort', OnWhat]], on=['Soort'], how='outer', suffixes=("_singlestoreclean", "_doelsysteem"))

    df = df.sort_values(OnWhat+ '_singlestore', ascending=False) 
    df.rename(columns={'Aantal_Records': 'Aantal_Records_doelsysteem'}, inplace=True)
    df = df.merge(df_soorten_meta, on=['Soort'], how='outer')
    df[lst_countColumns] = df[lst_countColumns].fillna(0)
    df[lst_countColumns] = df[lst_countColumns].astype(int, errors='ignore')

    df = df.style.bar(subset=lst_countColumns, color='#5fba7d', vmax=20000)
    
    return df

getOverview('Aantal_Records')

,Soort,Projecten,Aantal_Records_singlestore,Aantal_Records_singlestoreclean,Aantal_Records_doelsysteem
0,Standplaats,['MAGAZIJN'],8407,8407,0
1,Aardewerk,"['DC112', 'DB34', 'DC97', 'DC72', 'DC36']",6953,6953,0
2,Plaatsing,['MAGAZIJN'],5634,5634,5634
3,Vondst,"['DC112', 'DB34', 'DC97', 'DC93', 'DC154', 'DC179', 'DC11']",4521,4521,4521
4,Doos,['MAGAZIJN'],2773,2773,0
5,Spoor,"['DC179', 'DC97', 'DB34', 'DC93', 'DC154', 'DC112']",1826,1826,0
6,Foto,[nan],1405,1405,0
7,Project,['DELF-IT'],840,840,840
8,Hout,"['DC97', 'DC36', 'DB34', 'DC112', 'DC11', 'DC93']",480,480,0
9,Glas,"['DC93', 'DC97', 'DB34', 'DC154']",461,461,0


## Detailanalyse SingleStore naar Doelsysteem

In [8]:
df = df_singlestore.merge(df_doelsysteem, on=['Soort', 'Aantal_Records', 'Aantal_projecten'], how='outer', suffixes=("_singlestore", "_doelsysteem"))
df[df.Stage_singlestore.isnull() | df.Stage_doelsysteem.isnull() ]

,Soort,Projecten_singlestore,Aantal_Records,Aantal_projecten,Stage_singlestore,Projecten_doelsysteem,Stage_doelsysteem
0,Standplaats,[MAGAZIJN],8407,1,SingleStore,NaN,NaN
1,Aardewerk,"[DC112, DB34, DC97, DC72, DC36]",6953,5,SingleStore,NaN,NaN
4,Doos,[MAGAZIJN],2773,1,SingleStore,NaN,NaN
5,Spoor,"[DC179, DC97, DB34, DC93, DC154, DC112]",1826,6,SingleStore,NaN,NaN
6,Foto,[nan],1405,1,SingleStore,NaN,NaN
8,Hout,"[DC97, DC36, DB34, DC112, DC11, DC93]",480,6,SingleStore,NaN,NaN
9,Glas,"[DC93, DC97, DB34, DC154]",461,4,SingleStore,NaN,NaN
10,Keramiek,"[DC154, DC179]",418,2,SingleStore,NaN,NaN
11,Spijker,[DB34],400,1,SingleStore,NaN,NaN
12,Put,"[DC179, DC154]",345,2,SingleStore,NaN,NaN


In [9]:
s1 = set(['Aardewerk', 'Bot', 'Glas', 'Leer', 'Steen', 'Kleipijp', 'Skelet', 'Hout', 'Spijker', 'Muur', 'Keramiek', 'Metaal', 'Munt'])
s2 = set(['Put', 'Vlak', 'Spoor', 'Stelling', 'Aardewerk', 'Artefact', 'Standplaats', 'Project', 'Vindplaats', 'Vondst', 'Plaatsing', 'Doos'])

list(set.union(s1, s2))

['Keramiek',
 'Kleipijp',
 'Vindplaats',
 'Bot',
 'Leer',
 'Glas',
 'Standplaats',
 'Munt',
 'Hout',
 'Aardewerk',
 'Skelet',
 'Muur',
 'Spijker',
 'Spoor',
 'Vondst',
 'Project',
 'Stelling',
 'Steen',
 'Plaatsing',
 'Doos',
 'Put',
 'Artefact',
 'Vlak',
 'Metaal']

In [10]:
['Metaal',
 'Artefact',
 'Glas',
 'Stelling',
 'Hout',
 'Spijker',
 'Plaatsing',
 'Keramiek',
 'Put',
 'Vondst',
 'Standplaats',
 'Aardewerk',
 'Munt',
 'Muur',
 'Skelet',
 'Doos',
 'Project',
 'Spoor',
 'Vindplaats',
 'Leer',
 'Steen',
 'Bot',
 'Vlak',
 'Kleipijp']

['Metaal',
 'Artefact',
 'Glas',
 'Stelling',
 'Hout',
 'Spijker',
 'Plaatsing',
 'Keramiek',
 'Put',
 'Vondst',
 'Standplaats',
 'Aardewerk',
 'Munt',
 'Muur',
 'Skelet',
 'Doos',
 'Project',
 'Spoor',
 'Vindplaats',
 'Leer',
 'Steen',
 'Bot',
 'Vlak',
 'Kleipijp']